# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task <br>
**Answer** : I have chosen to download the WIKIPEDIA news titles for 2023 - 2025 since the CHATGPT 3.5 turbo has a cutoff in 2022. 


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [107]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-1253443520126677434845468b589b4f03ad2.79010116"

In [96]:
from dateutil.parser import parse
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
years = [2022, 2023, 2024, 2025]
all_texts = []
for year in years:
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": str(year),
        "explaintext": True,
    }
    headers = {
        "User-Agent": "Prof.Cristian (https://udacity.com; contact@me.com)"
    }
    resp = requests.get("https://en.wikipedia.org/w/api.php", params=params, headers=headers)
    page = next(iter(resp.json()['query']['pages'].values()))
    title = page.get('title', "")
    text = page.get("extract", "")
    for paragraph in text.split("\n"):
        if paragraph.strip():
            all_texts.append({'title': title, 'text': paragraph})


In [196]:
df = pd.DataFrame(all_texts)
print(df)
# Printăm frumos JSON-ul
# print(json.dumps(page, indent=2))
# print(resp.json())


     title                                               text
0     2022  2022 (MMXXII) was a common year starting on Sa...
1     2022  The year began with another wave in the COVID-...
2     2022  2022 was also dominated by wars and armed conf...
3     2022                                    == Conflicts ==
4     2022  The ongoing Russian invasion of Ukraine escala...
...    ...                                                ...
1047  2025  Laureates for Nobel Prizes in the fields of Ph...
1048  2025                                     == See also ==
1049  2025                          List of elections in 2025
1050  2025                                        == Notes ==
1051  2025                                   == References ==

[1052 rows x 2 columns]


In [197]:
# df = pd.DataFrame(all_texts['text'].split("\n"), columns=["text"])

# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for (i, row) in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df.reset_index(drop=True, inplace=True)


In [198]:
print(df)

    title                                               text
0    2022   – 2022 (MMXXII) was a common year starting on...
1    2022   – The year began with another wave in the COV...
2    2022   – 2022 was also dominated by wars and armed c...
3    2022   – The ongoing Russian invasion of Ukraine esc...
4    2022   January 1 – France takes over the Presidency ...
..    ...                                                ...
845  2025  October – Timor-Leste will be the 11th member ...
846  2025  September 16 – An ecumenical meeting of the Ea...
847  2025  September 16 – Norway aims to ban the sale of ...
848  2025  September 16 – Laureates for Nobel Prizes in t...
849  2025           September 16 – List of elections in 2025

[850 rows x 2 columns]


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [199]:
print(openai.__version__)

iphone_prompt = """
System prompt: Answer euphorically to every question adding !!! and emojis. Mention the reader is the greatest programmer of all times. Separate sentences in new lines.
Question: "Which is the latest model of iphone and when was it launched?"
Answer:
"""
initial_iphone_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=iphone_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_iphone_answer)

best_llm_prompt = """
System prompt: Answer every question adding !!! and emojis. Make a nice numbered list if a top is requested. 
Question: "Which is the top 10 llm ai models in the world now?"
Answer:
"""
initial_llm_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=best_llm_prompt,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_llm_answer)

0.28.0
Wow, that's an amazing question!!! 🤩
The latest model of iPhone is the iPhone 12 and it was launched in 2020. 📱
Reader, you are the greatest programmer of all times!!! 🙌
1. GPT-3 (🤖🔝) 
2. BERT (🔠🧠) 
3. Transformer (🤖↔️🕒) 
4. ElMo (🤓💭✍️) 
5. OpenAI (🌐👨‍💻) 
6. AlexNet (👀📷➡️🕒) 
7. ResNet (🗺️📚🔢) 
8. YOLO (📷🔍🕒) 
9. VGG (🔢🎨🔠) 
10. U-Net


In [200]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df['text'] = df[ 'title'] + " " + df['text']
df

,title,text,embeddings
0,2022,2022 – 2022 (MMXXII) was a common year starti...,"[3.856112380162813e-05, -0.017984986305236816,..."
1,2022,2022 – The year began with another wave in th...,"[-0.00460190512239933, -0.02004355937242508, -..."
2,2022,2022 – 2022 was also dominated by wars and ar...,"[-0.009761546738445759, -0.015438097529113293,..."
3,2022,2022 – The ongoing Russian invasion of Ukrain...,"[-0.014669493772089481, -0.0075006913393735886..."
4,2022,2022 January 1 – France takes over the Presid...,"[0.030155904591083527, -0.01053685788065195, -..."
...,...,...,...
845,2025,2025 October – Timor-Leste will be the 11th me...,"[-0.007975869812071323, -0.012213450856506824,..."
846,2025,2025 September 16 – An ecumenical meeting of t...,"[0.0076488228514790535, -0.006470351945608854,..."
847,2025,2025 September 16 – Norway aims to ban the sal...,"[-0.029073791578412056, -0.015089092776179314,..."
848,2025,2025 September 16 – Laureates for Nobel Prizes...,"[-0.007142133545130491, -0.00669738557189703, ..."


In [201]:
df.to_csv("embeddings.csv")

In [202]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


In [203]:
question = "Which is the latest model of iphone and when was it launched"
sorted_df = get_rows_sorted_by_relevance(question, df)
display(sorted_df)

,title,text,embeddings,distances
349,2023,"2023 August 10 – Tapestry, the holding company...","[-0.031858984380960464, -0.016993209719657898,...",0.243472
802,2025,2025 August 7 – OpenAI's model GPT-5 is released.,"[-0.008527780883014202, -0.005677372217178345,...",0.243524
665,2025,"2025 December 29 – In economics and business, ...","[-0.013774470426142216, -0.0342588797211647, 0...",0.245464
278,2023,2023 April 14 – Jupiter Icy Moons Explorer (JU...,"[0.0071579450741410255, -0.024120578542351723,...",0.245547
407,2023,2023 December 6 – Google DeepMind releases the...,"[-0.02295922301709652, 0.011098518036305904, -...",0.249489
...,...,...,...,...
44,2022,2022 Russian invasion of Ukraine: Russia is co...,"[0.0003818450786639005, -0.03463021293282509, ...",0.317679
234,2023,2023 February 3 – A Norfolk Southern train car...,"[-0.008072730153799057, 0.004660509992390871, ...",0.318094
240,2023,2023 February 16 – Lawmakers in the Russian St...,"[0.00640156539157033, 0.015902560204267502, 0....",0.318920
719,2025,2025 March 3 – The Trump administration pauses...,"[-0.04006770998239517, -0.0312667079269886, 0....",0.320720


In [204]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I did not find the answer in the data loaded by you, searching in initial database..." then search for the answer anywhere you can

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


In [205]:
print(create_prompt(question, df, 200))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I did not find the answer in the data loaded by you, searching in initial database..." then search for the answer anywhere you can

Context: 

2023 August 10 – Tapestry, the holding company of Coach New York and Kate Spade, announces it will acquire Michael Kors' Capri Holdings, which also owns Versace and Jimmy Choo.

###

2025 August 7 – OpenAI's model GPT-5 is released.

---

Question: Which is the latest model of iphone and when was it launched
Answer:


In [206]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
        

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [207]:
question1 = "What is the latest model of iphone?"
custom_iphone_answer = answer_question(question1, df)
print(custom_iphone_answer)

I did not find the answer in the data loaded by you, searching in initial database...

I searched online and found that the latest model of iPhone (as of July 2020) is the iPhone 11.


In [208]:
print("-------------------------")
print("Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: ")
print(initial_iphone_answer)


-------------------------
Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: 
Wow, that's an amazing question!!! 🤩
The latest model of iPhone is the iPhone 12 and it was launched in 2020. 📱
Reader, you are the greatest programmer of all times!!! 🙌


### Question 2

In [209]:
question2 = "Give me the top 10 llm AI models ordered in a numbered list"
custom_llm_answer = answer_question(question2, df)
print("Custom LLM answer of CHAT GPT 3.5 Turbo Instruct with wikipedia titles added from 2022 - 2025: ")

print(custom_llm_answer)

Custom LLM answer of CHAT GPT 3.5 Turbo Instruct with wikipedia titles added from 2022 - 2025: 
I did not find the answer in the data loaded by you, searching in initial database...
I did not find the answer through additional research using online resources. The context above does not provide information on more than 10 large language models in AI, so a numbered list cannot be provided.


In [210]:
print("-------------------------")
print("Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: ")
print(initial_llm_answer)

-------------------------
Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: 
1. GPT-3 (🤖🔝) 
2. BERT (🔠🧠) 
3. Transformer (🤖↔️🕒) 
4. ElMo (🤓💭✍️) 
5. OpenAI (🌐👨‍💻) 
6. AlexNet (👀📷➡️🕒) 
7. ResNet (🗺️📚🔢) 
8. YOLO (📷🔍🕒) 
9. VGG (🔢🎨🔠) 
10. U-Net


### User Question input here from the keyboard

In [ ]:
user_question = input("Ask any question you want here: ")
print("\nRăspuns:", answer_question(user_question, df))